In [ ]:
import re

import pandas as pd
import pyarrow.parquet as pq
import numpy as np

import pickle
import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

import tensorflow as tf

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

### 訓練用データセットを作成

In [ ]:
df = pd.read_csv("./data/kochi.csv")
df_tokyo_rain  = pd.read_csv("./data/tokyo.csv", usecols=["降水量"])
df["target"] = df_tokyo_rain["降水量"].apply(lambda x: 1 if x > 0 else 0)
df["target"] = df["target"].shift(-1)
df["降水量"] = df["降水量"].apply(lambda x: 1 if x > 0 else 0)

df = df.query("気圧 != 0.0").copy()
df = df.query("最高気温 != 0").copy()
df = df.query("最小湿度 != 0").copy()
df = df.query("平均湿度 != 0").copy()

In [ ]:
df.info()

In [ ]:
# 余分な記号や空白を除外する
def modify_string(x):
    x = re.sub(r'[\)|\]|\s]', '', x)
    return x

In [ ]:
df['最大風速（風向）'] = df['最大風速（風向）'].apply(modify_string)

In [ ]:
# 時間情報を三角関数で変換
def trig_encoding(df, col):
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / 12)
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / 12)
    # 不要列を削除
    df.drop(columns=[col], inplace=True)
    return df

In [ ]:
df = trig_encoding(df, "月")

In [ ]:
# One-Hot-Vector化
def one_hot_encoding(df, col):
    # One-Hot-Vectorの取得
    data_dummy = pd.get_dummies(df[col], drop_first=True, dtype='int64')
    print("カテゴリ特徴量")
    print(data_dummy.columns)

    # 元データと結合し不要列を削除する
    df = pd.concat([df, data_dummy], axis=1)
    df.drop(columns=[col], inplace=True)

    # 全カラムが数値型になっていることを確認する
    print("各特徴量の方を確認")
    print(df.dtypes)
    
    return df

In [ ]:
df = one_hot_encoding(df, "最大風速（風向）")

In [ ]:
# 不要列の削除
datasets = df.drop(columns=["年","日"])
# target列のNaNを0(2023/1/1の東京は降雨なし)で埋める
datasets['target'].fillna(0.0, inplace=True)

In [ ]:
# データセットの中身を確認
print(datasets.info())
datasets

### 分類器の実装

In [ ]:
# 連続値のデータのみ標準化を実施する
def stdnum(num_features, data):
    processor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_features)
        ],
        remainder='passthrough'
    )
    data_processed = processor.fit_transform(data)
    return data_processed

In [ ]:
# 目的変数と説明変数に分離してホールドアウト検証を実施
def split_variable(datasets, test_size, random_state):
    explains = datasets.columns[datasets.columns != 'target']
    X = datasets[explains]
    y = datasets["target"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, stratify=y, random_state=random_state)
    return X_train, X_test, y_train, y_test

In [ ]:
# モデルを保存する
def save_model(name, mod):
       with open(f"./models/pred_rain_{name}.model", "wb") as f:
              pickle.dump(mod, f)

In [ ]:
X_train, X_test, y_train, y_test = split_variable(datasets, 0.2, 0)

In [ ]:
num_features = ['気圧', '平均気温', '最高気温', '最低気温', '平均湿度', '最小湿度', '平均風速', '最大風速（風速）', '日照時間', '月_cos', '月_sin']
X_train_processed = stdnum(num_features,X_train)
X_test_processed = stdnum(num_features,X_test)

### ロジスティック回帰

In [ ]:
def objective(trial):
    model = LogisticRegression(
        C=trial.suggest_float('C', 1e-5, 1e+2, log=True),
        random_state=trial.suggest_int('random_state',0,200, log=False),
        max_iter=1500       
    )
    
    result = cross_validate(estimator=model, X=X_train_processed, y=y_train, cv=10, scoring='accuracy')
    val_accuracy = result['test_score'].mean()
    
    return val_accuracy

In [ ]:

# 最適化を実行
if __name__ == "__main__":
    # 実行ログを非表示
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    # studyの作成
    study = optuna.create_study(direction='maximize')
    
    # 最適化の実施
    study.optimize(objective, n_trials=100)

    # 最良のトライアルの確認
    print(" Best trial : ")
    best_trial = study.best_trial

    print(f" Value : {best_trial.value} ")

    print(" Params : ")
    for key, value in best_trial.params.items():
        print(f" {key} : {value} ")

In [ ]:
best_lr = LogisticRegression(C=best_trial.params['C'], random_state=best_trial.params['random_state'])
best_lr.fit(X_train_processed, y_train)
pred_test = best_lr.predict(X_test_processed)
accuracy_score(y_test, pred_test)

In [ ]:
# モデルの保存
save_model("lr", best_lr)

### Neural Network

In [ ]:
# 多層ニューラルネットワークモデル作成用の関数
def create_model(trial):
    
    # AdamWの学習率の探索範囲を設定
    optimeizer = tf.keras.optimizers.AdamW(
        learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True),
    )

    # 多層FNNの構築
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=28, name=f'hidden_1', activation='relu')
    ])
    for i in range(2,5):
        model.add(tf.keras.layers.Dense(units=100, name=f'hidden_{i}', activation='relu'))
    model.add(tf.keras.layers.Dense(units=1, name='output', activation='sigmoid'))
    model.build(input_shape=(None, 28))

    # 最適化手法、損失関数、評価指標の設定
    model.compile(optimizer=optimeizer,
                  loss = 'binary_crossentropy',
                  metrics = ["accuracy"])
    
    return model

In [ ]:
# ハイパーパラメータの指定（最適化対象外）
tf.random.set_seed(1)
EPOCHS = 100
BATCH_SIZE = 32

validation_split = 0.2
step_per_epoch = np.ceil(X_train_processed.shape[0] * (1-validation_split) / BATCH_SIZE)    
validation_steps = np.ceil(X_train_processed.shape[0] * validation_split / BATCH_SIZE)   

In [ ]:
# optunaにより最小化したい目的関数を準備
def objective(trial):
     
    # モデルの生成
    model = create_model(trial)
    
    # 過学習対策に EarlyStopping コールバックを設定。val_lossの値が３エポックに渡って改善されなかった場合に学習を中止する
    # 効率化のため TFKerasPruning コールバックを設定。精度が出る見込みが薄いハイパーパラメータの組み合わせについては早々に切り捨てる
    callbacks = [
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
        optuna.integration.TFKerasPruningCallback(trial, monitor='val_accuracy'),
    ]
    
    # モデルの訓練
    history = model.fit(X_train_processed, y_train,
                        validation_split=validation_split,
                        batch_size=BATCH_SIZE, 
                        epochs=EPOCHS, 
                        steps_per_epoch=step_per_epoch, 
                        validation_steps=validation_steps,
                        callbacks=callbacks,
                        verbose=0)
    
    # 最後の val_accuracy を出力
    return history.history['val_accuracy'][-1]

In [ ]:
# 最適化を実行
if __name__ == "__main__":
    # studyの作成。'枝刈り'の方法としてはMedianPrunerを設定
    study = optuna.create_study(
        direction='maximize', pruner=optuna.pruners.MedianPruner(n_startup_trials=2)
        )
    
    # 最適化の実施
    study.optimize(objective, n_trials=100)

    # 途中で枝刈りされたトライアルの数と、最後まで完了したトライアルの数を取得
    pruned_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[optuna.trial.TrialState.COMPLETE])
    
    # トライアル回数の確認
    print("Study statistics: ")
    print(f" Number of finished trials : {len(study.trials)} ")
    print(f" Number of pruned trials : {len(pruned_trials)} ")
    print(f" Number of complete trials : {len(complete_trials)} ")

    # 最良のトライアルの確認
    print(" Best trial : ")
    best_trial = study.best_trial

    print(f" Value : {best_trial.value} ")

    print(" Params : ")
    for key, value in best_trial.params.items():
        print(f" {key} : {value} ")

In [ ]:
# 最も正解率の高かったハイパーパラメータの組み合わせを用いてモデルを生成し訓練を実施する
best_nn = create_model(best_trial)

# 過学習対策で EarlyStopping を設定
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
]

# モデルの訓練
history = best_nn.fit(X_train_processed, y_train,
                         validation_split=validation_split,
                         batch_size=BATCH_SIZE, 
                         epochs=EPOCHS, 
                         steps_per_epoch=step_per_epoch, 
                         validation_steps=validation_steps,
                         callbacks=callbacks,
                         verbose=1)

In [ ]:
# テストデータを用いてモデルの汎化性能を評価する
test_eval = best_nn.evaluate(X_test_processed, y_test)
print('Test Acc :', test_eval[1])

In [ ]:
# モデルの保存
save_model("nn", best_nn)

### ランダムフォレスト

In [ ]:
def objective(trial):
     
     n_estimators = trial.suggest_int('n_estimators', 100, 1000)
     max_depth = trial.suggest_int('max_depth', 1, 32, log=True)
     max_features = trial.suggest_categorical('max_features', ['sqrt', 'log2'])
     criterion = trial.suggest_categorical('criterion', ["gini", "entropy"])
     random_state = random_state=trial.suggest_int('random_state',0,200, log=False)

     clf = RandomForestClassifier(n_estimators=n_estimators,
                                  max_depth=max_depth,
                                  max_features=max_features,
                                  criterion=criterion,
                                  random_state=random_state)

     result = cross_validate(estimator=clf, X=X_train, y=y_train, cv=10, scoring='accuracy')
     val_accuracy = result['test_score'].mean()
    
     return val_accuracy

In [ ]:

# 最適化を実行
if __name__ == "__main__":
    # 実行ログを非表示
    optuna.logging.set_verbosity(optuna.logging.WARNING)
    # studyの作成
    study = optuna.create_study(direction='maximize')
    
    # 最適化の実施
    study.optimize(objective, n_trials=100)

    # 最良のトライアルの確認
    print(" Best trial : ")
    best_trial = study.best_trial

    print(f" Value : {best_trial.value} ")

    print(" Params : ")
    for key, value in best_trial.params.items():
        print(f" {key} : {value} ")

In [ ]:
best_rfc = RandomForestClassifier(n_estimators=best_trial.params["n_estimators"],
                            max_depth=best_trial.params["max_depth"],
                            max_features=best_trial.params["max_features"],
                            criterion=best_trial.params["criterion"],
                            random_state=best_trial.params["random_state"])
best_rfc.fit(X_train, y_train)
pred_test = best_rfc.predict(X_test)
accuracy_score(y_test, pred_test)

In [ ]:
# モデルの保存
save_model("rfc", best_rfc)